In [2]:
#!pip install accelerate peft bitsandbytes pip install git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum --quiet

import argparse
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os
import pandas as pd

def prep_dataset(self,path):
    self.data_df =  pd.read_csv(path)
    self.data_df["text"] = self.data_df[["ocr_text", "response"]].apply(lambda x: "###Human: Summarize this following dialogue: " + x["ocr_text"] + "\n###Assistant: " +x["response"], axis=1)
    self.formatted_data = Dataset.from_pandas(self.data_df)

    return self.formatted_data


    
    
def prep_model(self,dataset_path,model_name_or_path,r,
                alpha,lora_dropout=0.1, bias="None",
                task_type="CASUAL_LM",target_modules=["q_proj", "v_proj"],
                output_dir="mistral-finetuned",
                per_device_train_batch_size=8,
                gradient_accumulation_steps=1,
                optim="paged_adamw_32bit",
                learning_rate=2e-4,
                lr_scheduler_type="cosine",
                save_strategy="epoch",
                logging_steps=100,
                num_train_epochs=1,
                max_steps=250,
                fp16=True,
                push_to_hub=False):
    self.formatted_data= self.prep_dataset(self.dataset_path)
    self.tokenizer= AutoTokenizer.from_pretrained(self.model_name_or_path)
    self.tokenizer.pad_token = self.tokenizer.eos_token
    self.quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=self.tokenizer)
    self.model = AutoModelForCausalLM.from_pretrained(
                                  self.model_name_or_path,
                                  quantization_config=self.quantization_config_loading,
                                  device_map="auto"
                            )
    
    self.model.config.use_cache=False
    self.model.config.pretraining_tp=1
    self.model.gradient_checkpointing_enable()

    self.model = prepare_model_for_kbit_training(self.model)

    self.peft_config = LoraConfig(
        r=self.r, lora_alpha=self.alpha, lora_dropout=self.lora_dropout, bias=self.bias, task_type=self.task_type, target_modules=self.target_modules
    )

    self.model = get_peft_model(self.model, self.peft_config)

    self.training_arguments = TrainingArguments(
        output_dir=self.output_dir,
        per_device_train_batch_size=self.per_device_train_batch_size,
        gradient_accumulation_steps=self.gradient_accumulation_steps,
        optim=self.optim,
        learning_rate=self.learning_rate,
        lr_scheduler_type=self.lr_scheduler_type,
        save_strategy=self.save_strategy,
        logging_steps=self.logging_steps,
        num_train_epochs=self.num_train_epochs,
        max_steps=self.max_steps,
        fp16=self.fp16,
        push_to_hub=False
    )

    self.trainer = SFTTrainer(
        model=self.model,
        train_dataset=self.formatted_data,
        peft_config=self.peft_config,
        dataset_text_field="text",
        args=self.training_arguments,
        tokenizer=self.tokenizer,
        packing=False,
        max_seq_length=1024
    )

    self.trainer.train()

    return None


